## Create pipeline
In the section we will be creating a pipeline using this `python-sdk` to harness the power of `Instill VDP`!

Before we get into this, please make sure a local instance of `Instill VDP` and `Instill Model` is running,  and the config file had been populated with the correct `url` and `api_token`

Let's get started!

### Import packages

In [1]:
from instill.clients import InstillClient

To setup your target instance at runtime, do the following at the very beginning of your script.

In [ ]:
from instill.configuration import global_config

global_config.set_default(
    url="api.instill.tech",
    token="instill_sk***",
    secure=True,
)

### Get the client

Get the unified client that connect to all the available services offered by `Instill VDP` and `Instill Model`, including
- mgmt_service
- pipeline_service
- model_service
- artifact_service

In [2]:
client = InstillClient()

user = client.mgmt_service.get_user()
# name: "users/admin"
# uid: "4767b74d-640a-4cdf-9c6d-7bb0e36098a0"
# id: "admin"
# ...
# ...

### Create a model

Now create a model `text-generation` in `Instill Model` for later use

In [ ]:
import instill.protogen.common.task.v1alpha.task_pb2 as task_interface
model_id = "model_text-generation"
client.model_service.create_model(
    model_id,
    task_interface.Task.TASK_TEXT_GENERATION,
    "REGION_GCP_EUROPE_WEST4",
    "CPU",
    "model-definitions/container",
    {},
)

### Build and deploy the model
`Instill Model` is an advanced MLOps/LLMOps platform that was specifically crafted to facilitate the efficient management and orchestration of model deployments for unstructured data ETL. With `Instill Model`, you can easily create, manage, and deploy your own custom models with ease in `Instill Core` or on the cloud with `Instill Cloud`.

Follow the instructions [here](https://www.instill.tech/docs/model/create) to build and deploy your model.


### Create a pipeline

Now we are ready to construct a pipeline!

The pipeline receipt below is a sample for demo. It simply returns the input string value.

In [5]:
pipeline_id = "pipeline_demo"
client.pipeline_service.create_pipeline(
    pipeline_id,
    "this is a pipeline for demo",
    {
        "output": {"result": {"title": "result", "value": "${variable.input}"}},
        "variable": {"input": {"instillFormat": "string", "title": "input"}},
    },
)

### Validate the pipeline

Before we trigger the pipeline, it is recommended to first validate the pipeline recipe first

In [ ]:
# validate the pipeline recipe
client.pipeline_service.validate_pipeline(pipeline_id)

### Trigger the pipeline

Now it is time to trigger this awesome pipeline!

In [10]:
client.pipeline_service.trigger_pipeline(pipeline_id, [], [{"input": "hello world"}])